In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("assets\metadata\Wrds-Refinitiv fields.xlsx", sheet_name='Conversion')
df

,Var Fields,Description,Global Compustat Annual,BB,BB Mnemonic,NOTE,Refinitiv Screener,Refinitiv Function,Refinitiv Output (GL file),Refinitiv Output (Modif)
0,ajex,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,prc_comp_adj,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,price,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,act,Current Assets - Total,act,NaN,NaN,NaN,Total Current Assets [STCA],TR.F.TotCurrAssets,Total Current Assets (USD) In the last 10 FY,Total Current Assets
4,ap,Accounts Payable - Trade,ap,NaN,NaN,NaN,Trade Account Payables - Total [SANP],TR.F.TradeAcctPbleTot,Trade Account Payables - Total (USD) In the la...,Trade Account Payables - Total
...,...,...,...,...,...,...,...,...,...,...
62,oepsx,Earnings Per Share - Diluted - from Operations,None,Diluted EPS from Continuing Operations - Adjusted,IS_DIL_EPS_CONT_OPS,"for North America, found on quarterly data;\ns...","EPS - Basic - excl Extraordinary Items, Common...",TR.F.EPSBasicExclExordItemsComTot,"EPS - Basic - excl Extraordinary Items, Common...","EPS - Basic - excl Extraordinary Items, Common..."
63,opeps,Earnings Per Share from Operations,None,Basic EPS from Continuing Operations,IS_EARN_BEF_XO_ITEMS_PER_SH,ok,EPS - Basic - excluding Extraordinary Items - ...,TR.F.EPSBasicExclExordItemsNormTot,EPS - Basic - excl Extraordinary Items - Norma...,EPS - Basic - excl Extraordinary Items - Norma...
64,oprepsx,Earnings Per Share - Diluted - from Operations,None,Diluted EPS from Continuing Operations - Adjusted,IS_DIL_EPS_CONT_OPS,similar to oepsx,Wrds: This is the annual ( company level ) Qua...,TR.F.EPSDilExclExordItemsNormTot,EPS - Diluted - excl Extraordinary Items - Nor...,EPS - Diluted - excl Extraordinary Items - Nor...
65,prcc_f,Price Close - Annual - Fiscal,None,Last Closing Price,"PX_LAST\n\nIf N/A, may be derived: HISTORICAL_...",ok,Price Close,TR.PriceClose,Price Close (USD) In the last 10 FY,Price Close


In [9]:
eu_df = pd.read_csv("assets\Eikon\europe_data_matched.csv", low_memory=False)
eu_df

,Instrument,TotAssets Date,TotAssets.Year,PriceClose Date,PriceClose.Year,Company Name,Country of Incorporation,Country of Exchange,Country of Headquarters,Instrument Type,...,Total Liabilities,Loans & Receivables - Total,Net Cash Flow from Operating Activities,Income Taxes - Payable - Short-Term,Trade Account Payables - Total,Advertising Expense,Labor & Related Expenses - Total,Dividend Per Share - Actual,"EPS - Basic - excl Extraordinary Items, Common - Total",Price Close
0,AMAV.VI,2021-12-31,2021.0,2021-12-30,2021.0,AMAG Austria Metall AG,Austria,Austria,Austria,Ordinary Shares,...,9.642860e+08,199173000.0,45592000.0,20947000.0,107860000.0,NaN,175154000.0,NaN,1.85112,41.00
1,AMAV.VI,2020-12-31,2020.0,2020-12-30,2020.0,NaN,NaN,NaN,NaN,NaN,...,9.455910e+08,151050000.0,107287000.0,3728000.0,59111000.0,NaN,146618000.0,0.5,0.31287,29.90
2,AMAV.VI,2019-12-31,2019.0,2019-12-30,2019.0,NaN,NaN,NaN,NaN,NaN,...,8.823700e+08,157929000.0,139943000.0,10331000.0,73050000.0,NaN,157819000.0,2.4,1.09579,30.50
3,AMAV.VI,2018-12-31,2018.0,2018-12-28,2018.0,NaN,NaN,NaN,NaN,NaN,...,9.403690e+08,164438000.0,94288000.0,75000.0,89966000.0,NaN,147939000.0,1.2,1.26307,31.20
4,AMAV.VI,2017-12-31,2017.0,2017-12-29,2017.0,NaN,NaN,NaN,NaN,NaN,...,7.969950e+08,153077000.0,101841000.0,1036000.0,77564000.0,NaN,140232000.0,1.2,1.79106,51.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272301,BUCN.S,2014-12-31,2014.0,2014-12-30,2014.0,NaN,NaN,NaN,NaN,NaN,...,1.402900e+09,583300000.0,163400000.0,23500000.0,263600000.0,NaN,710400000.0,6.5,18.58665,248.90
272302,BUCN.S,2013-12-31,2013.0,2013-12-30,2013.0,NaN,NaN,NaN,NaN,NaN,...,1.362200e+09,509500000.0,223800000.0,49500000.0,261200000.0,NaN,662800000.0,5.0,19.63819,259.00
272303,BUCN.S,2012-12-31,2012.0,2012-12-28,2012.0,NaN,NaN,NaN,NaN,NaN,...,1.369100e+09,476700000.0,200100000.0,42200000.0,238400000.0,NaN,630100000.0,4.0,15.52675,180.00
272304,BUCN.S,2011-12-31,2011.0,2011-12-30,2011.0,NaN,NaN,NaN,NaN,NaN,...,1.433300e+09,496500000.0,171500000.0,33300000.0,272800000.0,NaN,585200000.0,3.0,12.52053,164.20


In [12]:
for col in eu_df.columns:
    print(col)

Instrument
TotAssets Date
TotAssets.Year
PriceClose Date
PriceClose.Year
Company Name
Country of Incorporation
Country of Exchange
Country of Headquarters
Instrument Type
ISIN
ISIN Code
ISIN_CODE
SEDOL
CUSIP Code
SIC Industry Group Code
SIC Industry Group Name
SIC Industry Code
SIC Industry Name
GICS Sector Code
GICS Sector Name
GICS Industry Group Code
GICS Industry Group Name
GICS Industry Code
GICS Industry Name
GICS Sub-Industry Code
GICS Sub-Industry Name
CO2 Equivalent Emissions Direct, Scope 1
CO2 Equivalent Emissions Indirect, Scope 2
Upstream scope 3 emissions Purchased goods and services
Upstream scope 3 emissions Capital goods
Upstream scope 3 emissions Fuel- and Energy-related Activities
Upstream scope 3 emissions Transportation and Distribution
Upstream scope 3 emissions Waste Generated in Operations
Upstream scope 3 emissions Business Travel
Upstream scope 3 emissions Employee Commuting
Upstream scope 3 emissions Leased Assets
Downstream scope 3 emissions Transportation a

In [34]:
def eikon2wrds(df):
    # def ocf(df):
    #     a = coalesce(
    #         df['Net Cash Flow from Operating Activities'],
    #         (
    #             df['Net Income after Minority Interest'] -
    #             df['']
    #     )
    #     return a/b

    new_df = pd.DataFrame(dict(
        act=df['Total Current Assets'],
        ap=df['Trade Account Payables - Total'],
        at=df['Total Assets'],
        capx=df['Capital Expenditures - Total'],
        ceq=df['Common Equity Attributable to Parent Shareholders'],
        che=df['Cash & Short Term Investments - Total'],
        cogs=df['Cost Of Goods Sold - Actual'],
        dlc=df['Short-Term Debt & Current Portion of Long-Term Debt'],
        dltt=df['Debt - Long-Term - Total'],
        dp=df['Depreciation & Amortization - Supplemental'],
        dpc=df['Depreciation Depletion & Amortization - Cash Flow'],
        dvc=df['Dividends - Common - Cash Paid'],
        dvp=df['Dividends - Preferred - Cash Paid'],
        ebit=df['Earnings before Interest & Taxes (EBIT)'],
        ebitda=df['Operating Profit before Non-Recurring Income/Expense'],
        ib=df['Net Income after Minority Interest'],
        ibc=df['Income before Discontinued Operations & Extraordinary Items'],
        icapt=(
            df['Common Equity Attributable to Parent Shareholders'] + 
            df['Total Liabilities'] +
            df['Cash & Short Term Investments - Total']
        ),
        invt=df['Inventories - Total'],
        lct=df['Total Current Liabilities'],
        lt=df['Total Liabilities'],
        mib=df['Minority Interest - Equity'],
        mii=df['Minority Interest'],
        nopi=(
            df['Interest Income - Non-Bank'] + 
            df['Sale of Tangible & Intangible Fixed Assets - Gain/(Loss)'] +
            df['Equity Earnings/(Loss) before Taxes including Non-Recurring'] +
            df['Other Non-Operating Income/(Expense) - Total']
        ),
        oancf=df['Net Cash Flow from Operating Activities'],
        oiadp=df['Earnings before Interest & Taxes (EBIT)'],
        oibdp=df['Earnings before Interest Taxes Depreciation & Amortization'],
        pi=df['Income before Taxes'],
        ppent=df['Property Plant & Equipment - Net - Total'],
        rect=df['Loans & Receivables - Total'],
        revt=df['Revenue from Business Activities - Total'],
        sale=df['Revenue from Business Activities - Total'], ## CHECK
        seq=df["Shareholders' Equity - Attributable to Parent ShHold - Total"],
        spi=df['Non-Recurring Income/(Expense) - Total'],
        txdb=df['Deferred Tax - Liability - Long-Term'],
        txditc=df['Deferred Tax & Investment Tax Credits - Long-Term'],
        txp=df['Income Taxes - Payable - Short-Term'],
        txt=df['Income Taxes'],
        xido=df['Extraordinary Activities - after Tax - Gain/(Loss)'],
        xint=df['Interest Expense - Net of Capitalized Interest'],
        xlr=df['Labor & Related Expenses - Total'],
        xopr=(
            df['Selling General & Administrative Expenses - Total'] +
            df['Cost of Revenues - Unclassified']
        ),
        xrd=df['Research & Development Expense'],
        xsga=df['Selling General & Administrative Expenses - Total'],
        pstk=df['Preferred Shareholders Equity'],
        pstkl=df['Liquidation & Redemption Value of Redeem Pref Stock (Eq)'],
        pstkrv=df['Preferred Stock - Redeemable - Total'],
        be=df["Total Shareholders' Equity incl Minority Intr & Hybrid Debt"],
        csho=df['Common Shares - Outstanding - Total'],
        dvrate=df['Dividend Per Share - Actual'],
        epsfi=df['EPS - Diluted - incl Extraordinary Items, Common - Total'],
        epsfx=df['EPS - Diluted - excl Exord Items Applicable to Common Total'],
        gp=(
            df['Revenue from Business Activities - Total'] - 
            df['Cost of Revenues - Unclassified']
        ),
        ibadj=df['Income available to Common excluding Extraordinary Items'],
        ibcom=df['Income available to Common excluding Extraordinary Items'], ## CHECK
        itcb=(
            df['Deferred Tax & Investment Tax Credits - Long-Term'] - 
            df['Deferred Tax - Liability - Long-Term']
        ),
        mktcap=df['Market Capitalization'],
        ni=df['Net Income before Minority Interest'],
        # ocf=ocf, ## TODO
        oepsx=df['EPS - Basic - excl Extraordinary Items, Common - Total'],
        opeps=df['EPS - Basic - excl Extraordinary Items - Normalized - Total'],
        oprepsx=df['EPS - Diluted - excl Extraordinary Items - Normalized -Total'],
        prcc_f=df['Price Close'],
        xad=df['Advertising Expense']
    ))

    df = pd.concat([df.iloc[:, :27], new_df], axis=1)
    return df


new_df = eikon2wrds(eu_df)
new_df

,Instrument,TotAssets Date,TotAssets.Year,PriceClose Date,PriceClose.Year,Company Name,Country of Incorporation,Country of Exchange,Country of Headquarters,Instrument Type,...,ibadj,ibcom,itcb,mktcap,ni,oepsx,opeps,oprepsx,prcc_f,xad
0,AMAV.VI,2021-12-31,2021.0,2021-12-30,2021.0,AMAG Austria Metall AG,Austria,Austria,Austria,Ordinary Shares,...,65277000.0,65277000.0,0.0,1.445824e+09,64614000.0,1.85112,1.85109,1.85109,41.00,NaN
1,AMAV.VI,2020-12-31,2020.0,2020-12-30,2020.0,NaN,NaN,NaN,NaN,NaN,...,11033000.0,11033000.0,0.0,1.050274e+09,11059000.0,0.31287,0.32475,0.32475,29.90,NaN
2,AMAV.VI,2019-12-31,2019.0,2019-12-30,2019.0,NaN,NaN,NaN,NaN,NaN,...,38642000.0,38642000.0,0.0,1.075552e+09,38642000.0,1.09579,1.09579,1.09579,30.50,NaN
3,AMAV.VI,2018-12-31,2018.0,2018-12-28,2018.0,NaN,NaN,NaN,NaN,NaN,...,44541000.0,44541000.0,0.0,1.100237e+09,44541000.0,1.26307,1.26307,1.26307,31.20,NaN
4,AMAV.VI,2017-12-31,2017.0,2017-12-29,2017.0,NaN,NaN,NaN,NaN,NaN,...,63160000.0,63160000.0,0.0,1.812217e+09,63160000.0,1.79106,1.79106,1.79106,51.39,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272301,BUCN.S,2014-12-31,2014.0,2014-12-30,2014.0,NaN,NaN,NaN,NaN,NaN,...,187400000.0,187400000.0,0.0,2.551225e+09,189700000.0,18.58665,18.58665,18.50073,248.90,NaN
272302,BUCN.S,2013-12-31,2013.0,2013-12-30,2013.0,NaN,NaN,NaN,NaN,NaN,...,194500000.0,194500000.0,0.0,2.654750e+09,196200000.0,19.63819,19.63819,19.53222,259.00,NaN
272303,BUCN.S,2012-12-31,2012.0,2012-12-28,2012.0,NaN,NaN,NaN,NaN,NaN,...,151700000.0,151700000.0,0.0,1.845000e+09,156000000.0,15.52675,15.52675,15.45387,180.00,NaN
272304,BUCN.S,2011-12-31,2011.0,2011-12-30,2011.0,NaN,NaN,NaN,NaN,NaN,...,124700000.0,124700000.0,0.0,1.734921e+09,127400000.0,12.52053,12.52053,12.46448,164.20,NaN
